read in the ann files of the essay corpus and transform them into dataframes 

In [1]:
import pickle
import pandas as pd
import numpy as np
import re
from brat_parser import get_entities_relations_attributes_groups

In [2]:
##read in ANN files
ent_list=[]
rel_list=[]
att_list=[]
for i in range(1,103):
    
    try:
        if len(str(i))==1:
            string="00"+str(i)
        elif len(str(i))==2:
                string="0"+str(i)
        elif len(str(i))==3:
            string=str(i)
        else:
            raise ValueError
    except ValueError:
        print("The number"+str(i)+"is not expected. The number has to be between 0 and 999.")
    
    
    
   #input is split in the files between different annotations
    entities, relations, attributes, groups = get_entities_relations_attributes_groups("essays_all\essay"+string+".ann")
    
    ent_list.append(entities)
    rel_list.append(relations)
    att_list.append(attributes)
    

In [3]:
def get_relations_frame(relations):
    rel=pd.DataFrame(relations.items())
    rel=rel.drop([0],axis=1)

    x = rel.to_string(header=False,
                              index=False,
                              index_names=False).split('\n')

    rel_frame=pd.DataFrame([sub.split(",") for sub in x[:]])


    for i in range(len(rel_frame)):

        #simplify the strings by removing superflous information
        rel_frame.iloc[i,0]=re.search("'(.*)'",rel_frame.iloc[i,0]).group(1)

        rel_frame.iloc[i,1]=re.search("'(.*)'",rel_frame.iloc[i,1]).group(1)

        rel_frame.iloc[i,2]=re.search("'(.*)'",rel_frame.iloc[i,2]).group(1)

        rel_frame.iloc[i,3]=re.search("'(.*)'",rel_frame.iloc[i,3]).group(1)

    rel_frame=rel_frame.rename(columns={0: "ID", 1: "Stance", 2:"Targets", 3:"Supported"})

    return rel_frame

In [4]:
#get the entities frame
#Returns a Dataframe with the text the Target ID and the type of the Target (Claim,Prem,MJClaim)
def get_entities_frame(entities):
    
    ent=pd.DataFrame(entities.items())
    ent=ent.drop([0],axis=1)
    
    #split information into columns
    x = ent.to_string(header=False,
                          index=False,
                          index_names=False).split('\n')

    ent_frame=pd.DataFrame([sub.split(",") for sub in x[:]])
    
    #since the split happens along "," the text might be split over multiple columns
    #combine them again into one column
    ent_frame['text'] = ent_frame[ent_frame.columns[5:]].apply(
    lambda x: ','.join(x.dropna().astype(str)),
    axis=1
    )
    
    #move text column to front and drop a columns with text fragments form the back of the frame
    cols = ent_frame.columns.tolist()
    cols = cols[-1:] + cols[:-1]
    ent_frame = ent_frame[cols]

    ent_frame=ent_frame.drop(ent_frame.columns[3:],axis=1)

    #clean up text
    for i in range(len(ent_frame)):
        #ent_frame.iloc[i,0]=re.search("text='(.*)'",ent_frame.iloc[i,0]).group(1)
        ent_frame.iloc[i,1]=re.search("'(.*)'",ent_frame.iloc[i,1]).group(1)
        ent_frame.iloc[i,2]=re.search("'(.*)'",ent_frame.iloc[i,2]).group(1)


    ent_frame=ent_frame.rename(columns={0: "Targets", 1: "Type"})    

    return ent_frame



In [5]:
#get the attributes into a readable dataframe
def get_attribute_frame(attributes):
    #Transform the attributes into one dataframe
    att=pd.DataFrame(attributes.items())

    att=att.drop([0],axis=1)

    #get the values into a comma seperated list to make them accessible
    x = att.to_string(header=False,
                      index=False,
                      index_names=False).split('\n')

    vals = [','.join(ele.split()) for ele in x]
    
    #clean up dataframe
    attribute_frame=pd.DataFrame([sub.split(",") for sub in vals[:]])

    #rename columns
    attribute_frame=attribute_frame.rename(columns={0: "ID", 2: "Type", 4:"Targets",6:"Values"})

    
    
    for i in range(len(attribute_frame)):

        #simplify the strings by removing superflous information
        attribute_frame.iloc[i,0]=re.search("'(.*)'",attribute_frame.iloc[i,0]).group(1)

        attribute_frame.iloc[i,2]=re.search("'(.*)'",attribute_frame.iloc[i,2]).group(1)

        attribute_frame.iloc[i,4]=re.search("'(.*)'",attribute_frame.iloc[i,4]).group(1)

        attribute_frame.iloc[i,6]=re.search("'(.*)'",attribute_frame.iloc[i,6]).group(1)

    #drop empyt columns
    attribute_frame=attribute_frame.drop([1], axis=1)
    attribute_frame=attribute_frame.drop([3], axis=1)
    attribute_frame=attribute_frame.drop([5], axis=1)
    attribute_frame=attribute_frame.drop([7], axis=1)


   
    return attribute_frame

In [6]:
#convert into dataframes each list entry is one text 
att_frame_list=[]
ent_frame_list=[]
rel_frame_list=[]

for i,j,k in zip(att_list,ent_list,rel_list):
    att_frame_list.append(get_attribute_frame(i))
    ent_frame_list.append(get_entities_frame(j))
    rel_frame_list.append(get_relations_frame(k))

In [7]:

#pickle.dump(att_frame_list, open( "att_frame_list.p", "wb" ) )
#pickle.dump(ent_frame_list, open( "ent_frame_list.p", "wb" ) )
#pickle.dump(rel_frame_list, open( "rel_frame_list.p", "wb" ) )